In [1]:
%matplotlib widget
import numpy as np
import scipy
import obspy
from obspy.signal.cross_correlation import correlate, xcorr_max
from obspy.clients.fdsn.mass_downloader import RectangularDomain, Restrictions, MassDownloader
import matplotlib
import matplotlib.pyplot as plt
from matplotlib.widgets import Button
import matplotlib.patches as mpatches
import matplotlib.dates as mdates
from matplotlib.colors import LogNorm
from matplotlib.ticker import LogFormatterMathtext
from scipy.signal import spectrogram
import datetime
from geopy.distance import distance

In [ ]:
domain = RectangularDomain(minlatitude=-90, maxlatitude=90, minlongitude=-180, maxlongitude=180.0)

restrictions = Restrictions(
    starttime=obspy.UTCDateTime(2010, 1, 1),
    endtime=obspy.UTCDateTime(2022, 1, 1),
    chunklength_in_sec=86400,
    network="YT", station="UPTW,DNTW,BEAR,THUR", location="", channel="BH*",
    location_priorities = ["01",],
    channel_priorities = ["BHZ","BHN","BHE"],
    reject_channels_with_gaps=False,
    minimum_length=0.0)

mseed_storage=("/media/Data/Data/YT/MSEED/raw/")

mdl = MassDownloader(providers=["IRIS"])

mdl.download(
    domain=domain,
    restrictions=restrictions, 
    mseed_storage=mseed_storage, 
    stationxml_storage=("/media/Data/Data/YT/XML/"))


In [2]:
# set path and data parameters
path = "/media/Data/Data/YT/"
stats = ["BEAR","THUR","DNTW","UPTW"]
chans = "BH*"

# set path to XML response file
inv = obspy.Inventory()
for stat in stats:
    pathXML = path + "XML/" + "/*" + stat + ".xml"
    inv += obspy.read_inventory(pathXML)

In [ ]:
'''

Read data, remove instrumental response, and basic preprocessing for each window of interest constrained by imagery

'''

# set times of interest
time_win_dict = {}
time_win_dict[0] = {'start':datetime.datetime(2011,10,2),'end':datetime.datetime(2011,10,10)}
time_win_dict[1] = {'start':datetime.datetime(2015,7,8),'end':datetime.datetime(2015,7,20)}
time_win_dict[2] = {'start':datetime.datetime(2017,9,13),'end':datetime.datetime(2017,9,25)}
time_win_dict[3] = {'start':datetime.datetime(2018,9,21),'end':datetime.datetime(2018,10,25)}

# read in traces without instrument response removed
for i in range(len(time_win_dict)):
    duration = time_win_dict[i]['end']-time_win_dict[i]['start']
    files = []
    for day in range(0,duration.days):
        date = time_win_dict[i]['start']+datetime.timedelta(days=day)
        files.extend([path + "MSEED/raw/" + stat + "/*" + chans + "__" + str(date.year) + date.strftime('%m') + date.strftime('%d') + "T000000Z__" + "*" for stat in stats])
                      
    # read in raw data
    st_raw = obspy.Stream()
    for f in files:
        try:
            st_raw += obspy.read(f)
        except:
            print("No data for " + str(f))
                      
    # merge streams from corresponding stations and channels
    st_raw.merge()

    # basic preprocessing
    st_raw.detrend("demean")
    st_raw.detrend("linear")
    st_raw.taper(max_percentage=0.00025, max_length=20.)

    # remove response to lowest desired frequency
    st_1000s = st_raw.copy().remove_response(inventory=inv,pre_filt=[0.0005,0.001,15,20],output="VEL")

    # save full-resolution data
    starttime = time_win_dict[i]['start']
    endtime = time_win_dict[i]['end']
    start_string = str(starttime.year) + "-" + starttime.strftime('%m') + "-" + starttime.strftime('%d')
    end_string = str(endtime.year) + "-" + endtime.strftime('%m') + "-" + endtime.strftime('%d')
    st_1000s.write("data/MSEED/" + start_string + "_" + end_string + "_regional_velocity_1000s.MSEED")
    st_raw.write("data/MSEED/" + start_string + "_" + end_string + "_regional_velocity_raw.MSEED")

    # save downsampled data
    st_1000s_downsampled = st_1000s.resample(1)
    st_1000s_downsampled.write("data/MSEED/" + start_string + "_" + end_string + "_regional_velocity_1000s_downsampled.MSEED")
    st_raw_downsampled = st_raw.resample(1)
    st_raw_downsampled.write("data/MSEED/" + start_string + "_" + end_string + "_regional_velocity_raw_downsampled.MSEED")

No data for /media/Data/Data/YT/MSEED/raw/DNTW/*BH*__20111002T000000Z__*
No data for /media/Data/Data/YT/MSEED/raw/UPTW/*BH*__20111002T000000Z__*
No data for /media/Data/Data/YT/MSEED/raw/DNTW/*BH*__20111003T000000Z__*
No data for /media/Data/Data/YT/MSEED/raw/UPTW/*BH*__20111003T000000Z__*
No data for /media/Data/Data/YT/MSEED/raw/DNTW/*BH*__20111004T000000Z__*
No data for /media/Data/Data/YT/MSEED/raw/UPTW/*BH*__20111004T000000Z__*
No data for /media/Data/Data/YT/MSEED/raw/DNTW/*BH*__20111005T000000Z__*
No data for /media/Data/Data/YT/MSEED/raw/UPTW/*BH*__20111005T000000Z__*
No data for /media/Data/Data/YT/MSEED/raw/DNTW/*BH*__20111006T000000Z__*
No data for /media/Data/Data/YT/MSEED/raw/UPTW/*BH*__20111006T000000Z__*
No data for /media/Data/Data/YT/MSEED/raw/DNTW/*BH*__20111007T000000Z__*
No data for /media/Data/Data/YT/MSEED/raw/DNTW/*BH*__20111008T000000Z__*
No data for /media/Data/Data/YT/MSEED/raw/DNTW/*BH*__20111009T000000Z__*


In [ ]:
'''

Make plot for comparing data across stations and channels

'''

# get data
st_1000s = obspy.read("data/MSEED/2011-10-02_2011-10-09_regional_velocity_1000s.MSEED")
st_raw = obspy.read("data/MSEED/2011-10-02_2011-10-09_regional_velocity_raw.MSEED")

# set low cut period of data to be dispayed along with raw data
freq = [100,10,1]

# put data into dictionary
window_start = obspy.UTCDateTime(2011,10,2)
window_end = obspy.UTCDateTime(2011,10,10)
st_raw.trim(starttime=window_start,endtime=window_end)
st_1000s.trim(starttime=window_start,endtime=window_end)
first_stat = np.argmin([tr.stats.starttime for tr in st_1000s])
starttime = st_1000s[first_stat].stats.starttime
last_stat = np.argmax([tr.stats.endtime for tr in st_1000s])
endtime = st_1000s[last_stat].stats.endtime
fs = st_1000s[0].stats.sampling_rate
data_dict = {}
for tr in st_raw:
    key = tr.stats.station+tr.stats.channel+"raw"
    front_pad = tr.stats.starttime-starttime
    end_pad = endtime-tr.stats.endtime
    data = np.concatenate((np.zeros(int(np.floor(front_pad*fs))),tr.data,np.zeros(int(np.floor(end_pad*fs)))))
    #data = tr.data
    data_dict[key] = data*1000
for tr in st_1000s:
    for f in freq: 
        key = tr.stats.station+tr.stats.channel+str(f)
        tr_filt = tr.copy().filter("highpass",freq=1/f)
        front_pad = tr_filt.stats.starttime-starttime
        end_pad = endtime-tr_filt.stats.endtime
        data = np.concatenate((np.zeros(int(np.floor(front_pad*fs))),tr_filt.data,np.zeros(int(np.floor(end_pad*fs)))))
        #data = tr_filt.data
        data_dict[key] = data*1000

In [ ]:
# choose station and channel to plot
station = "BEAR"
channel = "BHZ"

# get time vector
t = st_raw[first_stat].times(type="matplotlib")
if len(data_dict[station+channel+"1"]) < len(t):
    t = t[:-1]

# make plot
nrows = len(freq)+1
ncols = 1
fig,ax = plt.subplots(nrows,ncols,figsize=[10,10])
plt.subplots_adjust(bottom=0.2)
colors = plt.rcParams['axes.prop_cycle'].by_key()['color']

# plot instrumental response
inv = obspy.read_inventory("data/XML/*.XML")
sampling_rate = st_raw[0].stats.sampling_rate
min_freq = 1/max(freq)
nfft = int(sampling_rate / min_freq)
response = inv.select(channel="BHZ",station="THUR")[0][0][0].response
cpx_response, freq_vector = response.get_evalresp_response(t_samp=1/sampling_rate, nfft=nfft)
ax[0].loglog(1/freq_vector,abs(cpx_response),'k')
ax[0].set(title="Instrument Response")
ax[0].set_xlabel("Period (s)")
ax[0].set_ylabel("Amplitiude")
ax[0].invert_xaxis()
box = ax[0].get_position()
box.y0 = box.y0 + 0.075
box.y1 = box.y1 + 0.075
ax[0].set_position(box)

# plot the data and set labels
for i in range(1,nrows):
    if i == 1:
        key = station+channel+str(freq[i-1])
        ax[i].set_title(station+" "+channel+"\n\n>"+str(freq[i-1])+"s",y=0.8)
        ax[i].get_xaxis().set_ticklabels([])
#         ax[1].set_ylabel("Counts")
    elif i == nrows-1:
        key = station+channel+str(freq[i-1])
        ax[i].set_title(">"+str(freq[i-1])+"s",y=0.8)
        ax[i].set_xlabel("Time")
        ax[i].xaxis.set_major_formatter(mdates.DateFormatter('%m/%d\n%H:%M'))
        ax[i].set_ylabel("Velocity (mm/s)")
    else:
        key = station+channel+str(freq[i-1])
        ax[i].set_title(">"+str(freq[i-1])+"s",y=0.8)
        ax[i].get_xaxis().set_ticklabels([])
        ax[i].set_ylabel("Velocity (mm/s)")
    ax[i].plot(t,data_dict[key],lw=2,color=colors[i-1])
    ax[i].set_xlim([t[0],t[-1]])
    ax[nrows-1].get_shared_x_axes().join(ax[nrows-1],ax[i])

plt.show()